# 第8章 函数中的类型提示

## 8.1 渐进式类型

- 渐进式类型系统的性质：
    1. 是可选的：默认情况下，类型检查工具不应对没有类型提示的代码发出警告。如果无法确定对象的类型时，会假定为Any类型。
    2. 不在运行时捕获类型错误：在运行时不能阻止把不一致的值传给函数或分配给变量。
    3. 不能改善性能。

## 8.2 类型由受支持的操作定义

- 鸭子类型：对象有类型，但变量（包括参数）没有类型。为对象声明的类型无关紧要，重要的是对象具体支持什么操作。比名义类型更灵活，但代价是运行时潜在的错误更多。
- 名义类型：对象只存在于运行时，类型检查工具只关心使用类型提示注解变量（包括参数）的源码。比鸭子类型更严格，优点是能在构建流水线中，甚至在IDE中输入代码的过程中更早地捕获一些bug。

## 8.3 注解中可用的类型

- typing.Any类型：动态类型
    1. 对T1及其子类型T2，T2与T1相容（里氏替换原则）
    2. 任何类型都与Any相容。
    3. Any与任何类型都相容。

- 简单的类型和类：例如int、float、str和bytes这样的简单的类型可以直接在类型提示中使用。

- Optional类型和Union类型：`Union[]`至少需要两种类型，嵌套的Union类型与扁平的Union类型效果相同。

- 泛化容器：泛型可以用类型参数来声明，以指定可以处理的项的类型。例如，`list[str]`。

- 元组类型：
    1. 用作记录的元组，例如：`tuple[str, float, str]`。
    2. 带有具名字段，用作记录的元组，使用`typing.NamedTuple`。
    3. 用作不可变序列的元组，例如：`tuple[Any,...]`。

- 泛化映射：使用`MappingType[KeyType, ValueType]`形式注解。

In [1]:
import sys
import re
import unicodedata
from collections.abc import Iterator

RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

def tokenize(text: str) -> Iterator[str]:  
    """返回全大写的单词构成的可迭代对象"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()

def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    index: dict[str, set[str]] = {}  
    for char in (chr(i) for i in range(start, end)):
        # 使用海象运算符，进行结果赋值
        if name := unicodedata.name(char, ''):  
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)
    return index

In [2]:
index = name_index(32, 65)
index['SIGN']

{'#', '$', '%', '+', '<', '=', '>'}

In [3]:
index['DIGIT']

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [4]:
index['DIGIT'] & index['EIGHT']

{'8'}

- 抽象基类：一般来说，在参数得类型提示中最好使用`abc.Mapping`或`abc.MutableMapping`，不要使用`dict`。

- Iterable：使用`collections.abc`包中的`Iterable`。

In [5]:
from collections.abc import Iterable

# 类型别名
FromTo = tuple[str, str]

def zip_replace(text: str, changes: Iterable[FromTo]) -> str: 
    for from_, to in changes:
        text = text.replace(from_, to)
    return text

In [6]:
l33t = [('a', '4'), ('e', '3'), ('i', '1'), ('o', '0')]
text = 'mad skilled noob powned leet'
zip_replace(text, l33t)

'm4d sk1ll3d n00b p0wn3d l33t'

- 参数化泛型和TypeVar：参数化泛型`list[T]`，其中`T`是类型变量，而密匙使用时会绑定具体的类型。

In [7]:
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

# 定义泛型参数
T = TypeVar('T')

def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]

- 静态协议（typing.Protocols）：协议通过`typing.Protocols`的子类定义，类型检查工具负责查找可用的协议类型，施行用法检查。

In [11]:
# 使用bound=SupportsLessThan的TypeVar定义top函数
from collections.abc import Iterable
from typing import TypeVar
from typing import Protocol, Any

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: 
        pass


LT = TypeVar('LT', bound=SupportsLessThan)

def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [12]:
top([4, 1, 5, 2, 6, 7, 3], 3)

[7, 6, 5]

In [13]:
l = 'mango pear apple kiwi banana'.split()
top(l, 3)

['pear', 'mango', 'kiwi']

- Callable：用于注解回调参数或高阶函数返回的可调用对象。

- NoReturn：仅用于注解绝不返回的函数的返回值类型。这类函数通常会抛出异常。

## 8.4 类型检查工具的缺点

- 一些便利的功能无法做静态检查，比如`config(**settings)`这种参数拆包。
- 对特性、描述符、元类和元编程等高级功能的支持很差，或者根本无法理解。
- 跟不上Python版本的变化，可能拒绝使用语言新特性的代码，甚至崩溃。